In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 

In [2]:
# 2014 
df = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_trip.tsv', sep='\t')
hh = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_household.tsv', sep='\t')
parcels = pd.read_csv(r'I:\T2040\soundcast_2014\inputs\accessibility\parcels_urbansim.txt', sep = ' ')
zone_tract = pd.read_csv(r'C:\Users\SChildress\Documents\Census Data\tract_zone.csv')
housing_cost_tract = pd.read_excel(r'C:\Users\SChildress\Documents\Census Data\MedianHousingCost_Tract.xlsx')
tractids = pd.read_csv(r'C:\Users\SChildress\Documents\Census Data\tract2010_nowater.csv')

In [3]:

# AAA Calculations availavble for small, medium, large Sedans (and average of these), SUV, Minivan

# Analysis of survey could be used to allocate vehicle type distributions?

# http://publicaffairsresources.aaa.biz/wp-content/uploads/2016/03/2016-YDC-Brochure.pdf
# results are averages for sedans

# Annualalize costs as 262
annual_factor = 262

# Costs by miles
# Operating costs total (gas + maintenance + tries)
depreciation_per_mile = 3759.0/15000    # depreciation by mile

# Annual costs
insurance = 1222    #(full-coverage insurance, license, registration, taxes, depreciation @ 15,000 miles per year), finance charge
taxes = 687    # license, registration, taxes
finance = 683    # load financing

annual_fixed_costs = insurance+taxes+finance

In [4]:
#to calculate activity durations for parking cost:
day_minutes = 1440
minutes_hr = 60.0
cents_dollar = 100

In [5]:
# get total travel cost (based on vmt)
# includes 
hh_mile_costs = df.groupby('hhno').sum()[['travcost']]
hh_mile_costs = hh_mile_costs.reset_index()

hh = pd.merge(hh, hh_mile_costs, on='hhno', how='left')

# Some households have 0 travel cost
hh['travcost'] = hh['travcost'].fillna(0)

In [6]:
# Total vehicle miles traveled per household to calculate depreciation
driver_trips = df[df['dorp'] == 1]

In [7]:
hh_travdist = df.groupby('hhno').sum()[['travdist']]
hh_travdist = hh_travdist.reset_index()

In [8]:
hh = pd.merge(hh, hh_travdist, on='hhno', how='left')

In [9]:
# annual operating costs
driver_trips['operating_cost_drivers'] = driver_trips['travcost']*annual_factor
driver_trips['travdist'] = driver_trips['travdist'].fillna(0)
driver_trips['depreciation'] = driver_trips['travdist']*depreciation_per_mile*annual_factor

#calculating parking costs based on activity duration
# the results look weird - add in parking cost later
#driver_trips = pd.merge(driver_trips, parcels[['PARCELID','PPRICHRP']], left_on='dpcl', right_on = 'PARCELID')
# if a trip spans midnight you need to adjust across the day - like this:
#driver_trips['duration'] = np.where(driver_trips['endacttm']<driver_trips['arrtm'], day_minutes-driver_trips['arrtm'] +driver_trips['endacttm'], driver_trips['endacttm']-driver_trips['arrtm'])/minutes_hr
#driver_trips['parking_cost'] = (driver_trips['duration']*driver_trips['PPRICHRP']*annual_factor)/cents_dollar



driver_trips_hh = driver_trips.groupby('hhno').sum()[['operating_cost_drivers','depreciation']].reset_index()
hh = pd.merge(hh, driver_trips_hh, on = 'hhno')

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [10]:
# fixed costs as function of number of vehicles owned
hh['veh_insurance'] = hh['hhvehs']*insurance
hh['veh_taxes'] = hh['hhvehs']*taxes
hh['veh_finance'] = hh['hhvehs']*finance

hh['annual_auto_costs'] = hh['operating_cost_drivers']+hh['depreciation']+hh['veh_insurance']+hh['veh_taxes']+hh['veh_finance']
hh['annual_auto_costs']=hh['annual_auto_costs'].fillna(0)
# Need to add per mile depreciation costs

In [11]:
# Calculate transit cost
transit_trips = df[df['mode'] == 6]

hh_travdist_transit_cost = transit_trips.groupby('hhno').sum()[['travcost']]
hh_travdist_transit_cost['transit_cost'] = hh_travdist_transit_cost['travcost']
hh_travdist_transit_cost = hh_travdist_transit_cost.reset_index()
hh_travdist_transit_cost['annual_transit_cost'] = hh_travdist_transit_cost['transit_cost']*annual_factor

In [12]:
hh = pd.merge(hh, hh_travdist_transit_cost[['hhno','annual_transit_cost']], on='hhno', how='left')
hh['annual_transit_cost'] = hh['annual_transit_cost'].fillna(0)

In [13]:
hh['total_cost']= hh['annual_transit_cost']+hh['annual_auto_costs']
hh = hh.loc[hh['hhincome']!=0]
hh['percent_transport_cost'] = hh['total_cost']/hh['hhincome']
hh['percent_transport_cost'] =hh['percent_transport_cost'].fillna(0)

In [14]:
print hh['total_cost'].mean()
print hh['veh_insurance'].mean()
print hh['veh_taxes'].mean()
print hh['operating_cost_drivers'].mean()
print hh['depreciation'].mean()
print hh['annual_transit_cost'].mean()
print hh['percent_transport_cost'].median()

11031.7978334
2466.04274411
1386.39227922
2586.48255642
3139.09778208
75.4623453191
0.133932544278


In [15]:
hh_costs = hh[['hhtaz','total_cost', 'veh_insurance', 'veh_taxes', 'operating_cost_drivers', 'depreciation', 'annual_transit_cost', 'percent_transport_cost']]

In [18]:
zone_tract.head()

,Unnamed: 0,taz_p,GEOID
0,0,1,53033000500
1,1,2,53033000402
2,2,3,53033000401
3,3,4,53033000401
4,4,5,53033000300


In [20]:
tractids.head()

,﻿STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,NAME_VAL,sqmiles,acres,geoidstr,perc,perc_1
0,53,33,22006,53033022006,220.06,Census Tract 220.06,G5020,S,1408907,0,47.704339,-122.199267,220.0,0.543948,348.126956,5.303302e+10,NaN,NaN
1,53,33,32320,53033032320,323.20,Census Tract 323.20,G5020,S,9554873,38494,47.756477,-122.124445,323.0,3.703739,2370.392801,5.303303e+10,NaN,NaN
2,53,33,31703,53033031703,317.03,Census Tract 317.03,G5020,S,7139218,643846,47.371689,-122.148431,317.0,3.005218,1923.339270,5.303303e+10,NaN,NaN
3,53,33,31206,53033031206,312.06,Census Tract 312.06,G5020,S,3668962,60430,47.313562,-122.183538,312.0,1.440042,921.626793,5.303303e+10,NaN,NaN
4,53,33,32102,53033032102,321.02,Census Tract 321.02,G5020,S,79040166,84391,47.485151,-121.965784,321.0,30.550272,19552.174321,5.303303e+10,NaN,NaN


In [24]:
# now join to the housing costs, but first we need to get the housing costs disaggregate to zones
zone_tractid = pd.merge(zone_tract,tractids, left_on= 'GEOID', right_on = 'GEOID10')
housing_cost_zone = pd.merge(housing_cost_tract, zone_tractid, left_on = 'Namelsad10', right_on = 'NAMELSAD10')

In [27]:
hh_costs = pd.merge(hh_costs, housing_cost_zone, left_on = 'hhtaz', right_on = 'taz_p')

In [28]:
hh_cost_taz = hh_costs.groupby('hhtaz').median()

In [31]:
hh_cost_taz['annual_median_housing_cost'] = hh_cost_taz['Median Monthly Housing Cost']*12

In [32]:
hh_cost_taz['housing_transport_cost'] = hh_cost_taz['annual_median_housing_cost']+hh_cost_taz['total_cost']

In [34]:
hh_cost_taz.to_csv(r'C:\Users\SChildress\Documents\travel-modeling\costs\hh_costs.csv')